In [1]:
import keras
from keras.models import Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def getInceptionV3():
    # get the inception V3 model
    modelInceptionV3 = keras.applications.InceptionV3(weights='imagenet')

    # remove the last layer
    modelInceptionV3OutLayer = modelInceptionV3.layers[len(modelInceptionV3.layers)-2].name
    modelInceptionV3NN =  Model(inputs=modelInceptionV3.input, outputs=modelInceptionV3.get_layer(modelInceptionV3OutLayer).output) 
    return modelInceptionV3NN

In [3]:
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
modelInceptionV3NN = getInceptionV3()

In [4]:
classes = np.loadtxt("/content/drive/My Drive/dl_replace_nn_output_cosine_sim/cancer/skinCancerdb.csv", delimiter=',') # skin cancer dataset

In [5]:
def get_class(value, maxval):
  if maxval < 0.8: # a high value cuz we better unk then fake m
    return 'unk - acc:' + str(maxval)
  if value in range(300):
    return 'M - acc:' + str(value) + str(maxval)
  if value in range(300, 608):
    return 'B - acc:' + str(value) + str(maxval)

In [6]:
def inferenceX(path_img):
  global classes
  global modelInceptionV3NN
  img_raw = tf.io.read_file(path_img)
  img = tf.io.decode_image(img_raw)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [299,299])
  img = np.expand_dims(img, 0)
  features = modelInceptionV3NN.predict(img)

  score = []
  for sample in classes:
    score.append(-1*cosine_loss(features, np.array([sample], dtype=np.float32)).numpy())
  max_s = max(score)
  return get_class(score.index(max_s), max_s), max_s

In [7]:
malign_benign = list() # [real class, predict]

In [8]:
contor = 300
for i in range(1,1501):
  try:
    malign_benign.append(['M',inferenceX('/content/drive/My Drive/dl_replace_nn_output_cosine_sim/cancer/test_s/malignant/'+str(i)+'.jpg')])
    contor = contor - 1
    if contor == 0:
      break
  except:
    continue

In [9]:
len(malign_benign)

300

In [10]:
contor = 300
for i in range(1,1801):
  try:
    malign_benign.append(['B',inferenceX('/content/drive/My Drive/dl_replace_nn_output_cosine_sim/cancer/test_s/benign/'+str(i)+'.jpg')])
    contor = contor - 1
    if contor == 0:
      break
  except:
    continue

In [11]:
len(malign_benign)

600

In [12]:
fake_benign = 0
continuee = 0
for i in range(0, 300):
  try:
    if malign_benign[i][0] != malign_benign[i][1][0][0]:
      if malign_benign[i][1][0][0] == 'u':
        continuee = continuee + 1
        continue
      fake_benign += 1
  except:
    continuee = continuee + 1
    continue

In [13]:
print(fake_benign)
print(continuee)

41
100


In [14]:
fake_malign = 0
continuee = 0
for i in range(300, 608):
  try:
    if malign_benign[i][0] != malign_benign[i][1][0][0]:
      if malign_benign[i][1][0][0] == 'u':
        continuee = continuee + 1
        continue
      fake_malign += 1
  except:
    continuee = continuee + 1
    continue

In [15]:
print(fake_malign)
print(continuee)

47
80


In [19]:
number_of_test = 600 - 100 - 80
number_of_worng = 41 + 47

In [20]:
ACC = 1 - number_of_worng / number_of_test

In [21]:
ACC

0.7904761904761904